In [1]:
cd ..

/home/studio-lab-user/sagemaker-studiolab-notebooks


In [2]:
cd openmic-2018

/home/studio-lab-user/sagemaker-studiolab-notebooks/openmic-2018


In [3]:
#!pip install wandb

In [4]:
import json
import os
import numpy as np
#!pip install pandas
import pandas as pd
import tensorflow as tf
#import wandb
#from wandb.keras import WandbCallback

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import multilabel_confusion_matrix, classification_report

2022-11-06 07:45:54.837205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-06 07:45:55.780577: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-06 07:45:55.780602: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-06 07:45:55.898617: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-06 07:45:57.725536: W tensorflow/stream_executor/pla

In [5]:
data = np.load("openmic-2018.npz", allow_pickle = True)
X, Y_true, Y_mask, sample_key = data['X'], data['Y_true'], data['Y_mask'], data['sample_key']
print(X.shape, Y_true.shape, Y_mask.shape, sample_key.shape)

(20000, 10, 128) (20000, 20) (20000, 20) (20000,)


In [6]:
print(Y_true[0])
Y_true1 = np.where(Y_true > 0.5, 1, 0)
print(Y_true1[0])

[0.5     0.5     0.5     0.5     0.17105 0.5     0.5     0.      0.5
 0.5     0.5     0.5     0.5     0.5     0.5     0.5     0.      0.5
 0.5     0.5    ]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [7]:
IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model

In [8]:
def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

In [9]:
BATCH_SIZE = 32 # Big enough to measure an F1-score
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time
SHUFFLE_BUFFER_SIZE = 128 # Shuffle the training data by a chunck of 128 observations

In [10]:
def create_dataset(filenames, labels, is_training=True):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    
    # if is_training == True:
    #     # This is a small dataset, only load it once, and keep it in memory.
    #     dataset = dataset.cache()
    # Shuffle the data each buffer size
    # dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [11]:
# PROLLY DONT NEED THIS
# Load training split csv file
with open("partitions/split01_train.csv") as f:
    train_IDs = f.readlines()
    train_IDs = np.array([ID.strip() for ID in train_IDs])
print(train_IDs.shape)

# Load test split csv file
with open("partitions/split01_test.csv") as f:
    test_IDs = f.readlines()
    test_IDs = np.array([ID.strip() for ID in test_IDs])
print(test_IDs.shape)

# Get the training and testing split data into np arrays
# Creates an array, which contains the index of the sample in data.  
train_index = np.array([i for i in range(20000) if sample_key[i] in train_IDs])
test_index = np.array([i for i in range(20000) if sample_key[i] in test_IDs])

print(train_index.shape)
print(test_index[1:10])

(14915,)
(5085,)
(14915,)
[29 30 31 32 33 34 35 36 37]


In [12]:
train_paths = [f"/home/studio-lab-user/sagemaker-studiolab-notebooks/openmic-2018/audio-logmel/{i[:3]}/{i}.png" for i in train_IDs]
print(train_paths[1])

test_paths = [f"/home/studio-lab-user/sagemaker-studiolab-notebooks/openmic-2018/audio-logmel/{i[:3]}/{i}.png" for i in test_IDs]
print(test_paths[1])

train_labels = [Y_true1[i] for i in train_index]
print(train_labels[1])

test_labels = [Y_true1[i] for i in test_index]
print(test_labels[1])

/home/studio-lab-user/sagemaker-studiolab-notebooks/openmic-2018/audio-logmel/000/000135_483840.png
/home/studio-lab-user/sagemaker-studiolab-notebooks/openmic-2018/audio-logmel/000/000308_61440.png
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
x_train = create_dataset(train_paths, train_labels)
print(x_train)

x_test = create_dataset(test_paths, test_labels)
print(x_test)

2022-11-06 07:46:05.186410: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-06 07:46:05.187222: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-06 07:46:05.187263: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2022-11-06 07:46:05.193611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 20), dtype=tf.int64, name=None))>
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 20), dtype=tf.int64, name=None))>


In [14]:
for element in x_train.as_numpy_iterator():
    print(element[1].shape)
    break

(32, 20)


In [15]:
# import necessary layers  
from tensorflow.keras.layers import Input, Conv2D 
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense 
from tensorflow.keras import Model
# input

input = Input(shape =(224,224,3))
# 1st Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(input)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

# 2nd Conv Block
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

# 3rd Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

# 4th Conv block
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

# Fully connected layers
x = Flatten()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 1024, activation ='relu')(x)
x = Dense(units = 512, activation ='relu')(x)
x = Dense(units = 256, activation ='relu')(x)
x = Dense(units = 128, activation ='relu')(x)
output = Dense(units = 20, activation ='sigmoid')(x)

# creating the model

model = Model (inputs=input, outputs =output)
model.summary()


2022-11-06 07:46:06.299698: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1644167168 exceeds 10% of free system memory.
2022-11-06 07:46:07.492275: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1644167168 exceeds 10% of free system memory.
2022-11-06 07:46:07.882674: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1644167168 exceeds 10% of free system memory.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 256)       295168

In [16]:
LR = 1e-5 # Keep it small when transfer learning
EPOCHS = 2
loss_fn = tf.keras.losses.BinaryCrossentropy()

In [17]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss=loss_fn, metrics = ['Recall', 'Precision'])

In [18]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

history = model.fit(x_train, epochs = EPOCHS, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_data = x_test)

Epoch 1/2


2022-11-06 07:46:08.797681: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1644167168 exceeds 10% of free system memory.
2022-11-06 07:46:09.137425: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1644167168 exceeds 10% of free system memory.


  5/467 [..............................] - ETA: 1:27:05 - loss: 0.6839 - recall: 0.2167 - precision: 0.0347

KeyboardInterrupt: 

In [ ]:
# list all data in history
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
# plt.plot(history.history['recall'])
# plt.plot(history.history['precision'])
# plt.title('train precision-recall')
# plt.ylabel('precision')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(x_test, batch_size = 1)
print(y_pred[0])
y_pred1 = np.where(y_pred > 0.5, 1, 0)
print(y_pred1[0])

In [ ]:
print(multilabel_confusion_matrix(test_labels, y_pred1))

In [ ]:
print(classification_report(test_labels, y_pred1))

# Load model and get classification report

In [ ]:
from keras.models import load_model

In [ ]:
cd openmic-2018

In [ ]:
model = load_model('mdl_wts.hdf5')

In [ ]:
y_pred = model.predict(x_test, batch_size = 1)
print(y_pred[0])
y_pred1 = np.where(y_pred > 0.5, 1, 0)
print(y_pred1[0])

In [ ]:
print(classification_report(test_labels, y_pred1))